In [ ]:
!nvidia-smi

Clone threestudio repo

In [ ]:
!git clone https://github.com/threestudio-project/threestudio.git

In [ ]:
%cd threestudio

Install dependencies

In [ ]:
!pip install ninja
!pip install --pre -U xformers
!pip install lightning==2.0.0 omegaconf==2.3.0 jaxtyping typeguard diffusers transformers accelerate opencv-python tensorboard matplotlib imageio imageio[ffmpeg] trimesh bitsandbytes sentencepiece safetensors huggingface_hub libigl xatlas 

And build some dependencies manually. This may take a while.

In [ ]:
!pip install git+https://github.com/KAIR-BAIR/nerfacc.git@v0.5.2
!pip install git+https://github.com/NVlabs/nvdiffrast.git
!pip install git+https://github.com/NVlabs/tiny-cuda-nn/#subdirectory=bindings/torch

Login to huggingface to 

In [ ]:
from huggingface_hub import login

login()

Now create your own 3D model from text prompts

In [ ]:
!python launch.py --config configs/dreamfusion-if.yaml --train --gpu 0 system.prompt_processor.prompt="a zoomed out DSLR photo of a baby bunny sitting on top of a stack of pancakes"

Display the rendered video

In [ ]:
from IPython.display import HTML
from base64 import b64encode
def display_video(video_path):
  mp4 = open(video_path,'rb').read()
  data_url = "data:video/mp4;base64," + b64encode(mp4).decode()
  return HTML("""
  <video width=1000 controls>
    <source src="%s" type="video/mp4">
  </video>
  """ % data_url)

In [ ]:
# replace the video_path here
save_dir = '/content/threestudio/outputs/dreamfusion-if/a_zoomed_out_DSLR_photo_of_a_baby_bunny_sitting_on_top_of_a_stack_of_pancakes@20230511-093842'

video_path = f'{save_dir}/save/it10000-test.mp4'
display_video(video_path)

Then extract the object mesh, specify `system.exporter.context_type=cuda` on colab. You can first try `system.geometry.isosurface_threshold=auto` and visualize it. Then you can adjust the threshold according to the auto value and the visual quality. Increase it if too many floaters and decrease it if incomplete geometry. Each run takes around 2 mins on T4.

In [ ]:
!python launch.py --config $save_dir/configs/parsed.yaml --export --gpu 0 resume=$save_dir/ckpts/last.ckpt system.exporter_type=mesh-exporter system.exporter.context_type=cuda system.geometry.isosurface_threshold=15.0 

For visualizing obj in colab, using open3d and plotly. Or you can directly download the obj file and check it locally.

In [ ]:
!pip install open3d plotly

In [ ]:
import numpy as np
import open3d as o3d
import plotly.graph_objects as go
mesh = o3d.io.read_triangle_mesh(f"{save_dir}/save/it10000-export/model.obj")
if not mesh.has_vertex_normals():
  mesh.compute_vertex_normals()
if not mesh.has_triangle_normals():
  mesh.compute_triangle_normals()

triangles = np.asarray(mesh.triangles)
vertices = np.asarray(mesh.vertices)
colors = None
if mesh.has_triangle_normals():
  colors = (0.5, 0.5, 0.5) + np.asarray(mesh.triangle_normals) * 0.5
  colors = tuple(map(tuple, colors))
else:
  colors = (1.0, 0.0, 0.0)
fig = go.Figure(
  data=[
    go.Mesh3d(
      x=vertices[:,0],
      y=vertices[:,1],
      z=vertices[:,2],
      i=triangles[:,0],
      j=triangles[:,1],
      k=triangles[:,2],
      facecolor=colors,
      opacity=0.50)
  ],
  layout=dict(
    scene=dict(
      xaxis=dict(visible=False),
      yaxis=dict(visible=False),
      zaxis=dict(visible=False)
    )
  )
)
fig.show()
